In [22]:

from requests import post
from pydantic import BaseModel
from trilogy import Dialects


In [24]:
class ModelSourceInSchema(BaseModel):
    alias: str
    contents: str


class ModelInSchema(BaseModel):
    name: str
    sources: list[ModelSourceInSchema]
    

class QueryInSchema(BaseModel):
    query: str
    dialect:Dialects
    full_model: ModelInSchema | None = None

In [33]:
import datetime

request = QueryInSchema(
    query = '''
select 1-> test;''',
    dialect='bigquery'
)
start = datetime.datetime.now()
print(request.model_dump_json())
x = post('http://localhost:5678/generate_query', data=request.model_dump_json()).json()
print(datetime.datetime.now() - start)
print(x)
print(x['generated_sql'])


{"query":"\nselect 1-> test;","dialect":"bigquery","full_model":null}
0:00:02.167828
{'generated_sql': '\nSELECT\n    1 as `test`\n\n\n', 'columns': [{'name': 'local.test', 'datatype': 'int', 'purpose': 'const'}]}

SELECT
    1 as `test`



